In [ ]:
from flask import Flask, render_template, request
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import os
from google.colab import output  # For exposing the server in Colab
from werkzeug.middleware.proxy_fix import ProxyFix

# Initialize Flask app
app = Flask(__name__)
app.template_folder = "/content/templates"

# Load ResNet50 model and adapt for binary classification
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')  # Load pre-trained model
# Add a custom dense layer for binary classification
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

model = Sequential([
    base_model,
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define labels
LABELS = ["non_damaged", "damaged"]

@app.route('/', methods=['GET'])
def index():
    # Render the upload page
    return render_template('index.html')

@app.route('/', methods=['POST'])
def predict():
    # Get the uploaded image
    imagefile = request.files['imagefile']
    if not imagefile:
        return "No image file uploaded!", 400

    # Save the uploaded image
    upload_folder = "/content/uploads"
    os.makedirs(upload_folder, exist_ok=True)  # Ensure the folder exists
    image_path = os.path.join(upload_folder, imagefile.filename)
    imagefile.save(image_path)

    # Preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize image to ResNet50 input size
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))  # Reshape for the model
    image = preprocess_input(image)  # Preprocess for ResNet50

    # Make prediction
    yhat = model.predict(image)
    confidence = yhat[0][0]  # Model's confidence (probability for class "damaged")
    label_index = int(confidence > 0.5)  # Threshold of 0.5 for binary classification
    classification = LABELS[label_index]  # Map prediction to label

    # Format prediction with percentage
    confidence_percentage = confidence_percentage = confidence * 100 if classification == "damaged" else (1 - confidence) * 100


    formatted_prediction = f"{classification} ({confidence_percentage:.2f}%)"

    #  Render the result
    return render_template('index.html', prediction=formatted_prediction)

if __name__ == '__main__':
    # Fix for running behind a proxy (like in Colab)
    app.wsgi_app = ProxyFix(app.wsgi_app, x_for=1, x_proto=1, x_host=1, x_prefix=1)

    # Expose the server in Colab
    port = 5000
    output.serve_kernel_port_as_iframe(port)
    app.run(host='0.0.0.0', port=port)


<IPython.core.display.Javascript object>

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Feb/2025 15:32:41] "GET /?authuser=0 HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [14/Feb/2025 15:32:55] "POST /?authuser=0 HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


INFO:werkzeug:127.0.0.1 - - [14/Feb/2025 15:33:16] "POST /?authuser=0 HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


INFO:werkzeug:127.0.0.1 - - [14/Feb/2025 15:33:32] "POST /?authuser=0 HTTP/1.1" 200 -
